In [1]:
import os
import torch
import numpy as np
import pandas as pd
import random
from torch.utils.data import DataLoader, Dataset
import math

class SpeedDataset(Dataset):
    def __init__(self, directory, sequence_length=5, augment=False, overlap=1):
        self.data = []
        self.sequence_length = sequence_length
        self.augment = augment  # Enable or disable augmentation
        self.overlap = overlap  # Control the overlap between sequences
        self.preprocess_data(directory)

    def preprocess_data(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.csv'):
                filepath = os.path.join(directory, filename)
                df = pd.read_csv(filepath)
                df.sort_values(by=['id', 'frame'], inplace=True)  # Sort by track ID and frame number
                
                track_data = df.groupby('id')
                
                for track_id, group in track_data:
                    group = group.reset_index(drop=True)  # Reset index after sorting
                    for i in range(0, len(group) - self.sequence_length + 1, self.overlap):
                        # Check if the frames within the window are consecutive
                        if group['frame'].iloc[i:i + self.sequence_length].diff().dropna().eq(1).all():
                            frames = group.iloc[i:i + self.sequence_length]
                            features = self.extract_features(frames[['x_tl_2d', 'y_tl_2d', 'x_br_2d', 'y_br_2d']].values)
                            speed = frames['average_speed_5_frames'].mean()  # Calculate the average speed for the sequence
                            self.data.append((features, speed))

    def extract_features(self, frames):
        features = []
        for i in range(1, len(frames)):
            current_frame = frames[i]
            previous_frame = frames[i-1]
            features.append(self.compute_frame_features(current_frame, previous_frame))
        return features

    def compute_frame_features(self, current_frame, previous_frame):
        x1, y1, x2, y2 = current_frame
        px1, py1, px2, py2 = previous_frame
        
        x1_change = abs(x1 - px1)
        y1_change = abs(y1 - py1)
        x2_change = abs(x2 - px2)
        y2_change = abs(y2 - py2)

        center_x_change = abs((x1 + x2)/2 - (px1 + px2)/2)
        center_y_change = abs((y1 + y2)/2 - (py1 + py2)/2)
        distance_moved1 = math.sqrt(center_x_change ** 2 + center_y_change ** 2)
        distance_moved2 = math.sqrt(x1_change ** 2 + y1_change ** 2)
        distance_moved3 = math.sqrt(x2_change ** 2 + y2_change ** 2)

        feature_vector = [distance_moved1, distance_moved2, distance_moved3, center_x_change, center_y_change, x1_change, y1_change, x2_change, y2_change]
        return feature_vector

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, output = self.data[idx]
        if self.augment:
            inputs = self.apply_augmentation(inputs)
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(output, dtype=torch.float32)

    def apply_augmentation(self, inputs):
        if random.random() > 0.6:
            inputs = self.add_noise(inputs)
        if random.random() > 0.99:
            inputs = self.time_shift(inputs, shift=random.choice([-1, 1]))
        if random.random() > 0.8:
            inputs = self.scale_features(inputs, scale=random.uniform(0.9, 1.1))
        if random.random() > 0.99:
            inputs = self.mirror_features(inputs)
        return inputs

    def add_noise(self, features, noise_level=0.05):
        noise = np.random.normal(0, noise_level, features.shape)
        return features + noise

    def time_shift(self, features, shift=1):
        if shift > 0:
            return np.vstack([np.zeros((shift, features.shape[1])), features[:-shift]])
        elif shift < 0:
            return np.vstack([features[-shift:], np.zeros((-shift, features.shape[1]))])
        return features

    def scale_features(self, features, scale=1.1):
        return features * scale

    def mirror_features(self, features):
        features_copy = features.copy()
        features_copy[:, [0, 2]] = -features_copy[:, [0, 2]]  # Assume these indices are the x-coordinates
        return features_copy


In [2]:
print(random.random())

0.7310559572644723


In [3]:
import torch

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attention = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        scores = self.attention(x).squeeze(2) 
        alpha = F.softmax(scores, dim=1).unsqueeze(2)  
        context = (x * alpha).sum(dim=1)
        return context, alpha


In [41]:
class SpeedPredictor(nn.Module):
    def __init__(self, sequence_length, feature_size, embedding_dim, hidden_dim, output_size):
        super(SpeedPredictor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=feature_size, out_channels=hidden_dim, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=embedding_dim, out_channels=hidden_dim, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.lstm = nn.RNN(hidden_dim, hidden_dim, batch_first=True, num_layers=4, dropout=0.2)
        self.attention = Attention(hidden_dim)
        self.conv3 = nn.Conv1d(in_channels=hidden_dim, out_channels=64, kernel_size=1)
        self.conv4 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=1)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(32, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)  
        x = self.conv1(x)
        #x = self.conv2(x)
        x = self.bn1(x)
        x = x.permute(0, 2, 1) 
        x, _ = self.lstm(x)
        x, attn_weights = self.attention(x)  
        x = x.unsqueeze(2) 
        x = self.dropout(x)
        x = self.conv3(x)
        x = x.squeeze(2)  
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
def init_weights(m):
    if type(m) == nn.LSTM:
        torch.nn.init.xavier_uniform_(m.weight_ih_l0)
        torch.nn.init.orthogonal_(m.weight_hh_l0)
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)


In [42]:
def train_and_validate(model, train_loader, val_loader, criterion, optimizer,scheduler, epochs):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        # Training
        model.train()
        total_train_loss = 0
        for inputs, speeds in train_loader:
            inputs, speeds = inputs.to(device), speeds.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.squeeze() 
            if outputs.shape != speeds.shape:
                outputs = outputs[:speeds.size(0)]  
            loss = criterion(outputs, speeds)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        scheduler.step()
        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for inputs, speeds in val_loader:
                inputs, speeds= inputs.to(device), speeds.to(device)
                outputs = model(inputs)
                outputs = outputs.squeeze()  
                if outputs.shape != speeds.shape:
                    outputs = outputs[:speeds.size(0)]  
                loss = torch.sqrt(criterion(outputs, speeds))
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        if epoch % 10 ==0:
            print(f'Epoch {epoch+1}/{epochs}, MSE training Loss: {avg_train_loss:.4f}, RMSE validation Loss: {avg_val_loss:.4f}')

    return train_losses, val_losses


In [12]:
import torch
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ExponentialLR

train_dataset = SpeedDataset('../train', sequence_length=10, overlap = 5, augment =False)
test_dataset = SpeedDataset('../val', sequence_length=10, overlap = 5)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
len(train_dataset)

25978

In [ ]:
# Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SpeedPredictor(sequence_length=10, feature_size=9, embedding_dim=64, hidden_dim=128, output_size=1)
model.to(device)
model.apply(init_weights)
criterion = nn.MSELoss()
optimizer = SGD(model.parameters(), lr=0.0005)

# Scheduler for learning rate adjustment
scheduler = ExponentialLR(optimizer, gamma=0.9992)
train_losses,val_losses = train_and_validate(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=200)

Epoch 1/200, MSE training Loss: 185.9566, RMSE validation Loss: 13.0377
Epoch 11/200, MSE training Loss: 81.9152, RMSE validation Loss: 13.1224
Epoch 21/200, MSE training Loss: 72.3270, RMSE validation Loss: 11.4317
Epoch 31/200, MSE training Loss: 64.9158, RMSE validation Loss: 12.5526
Epoch 41/200, MSE training Loss: 60.6061, RMSE validation Loss: 12.1434
Epoch 51/200, MSE training Loss: 53.9000, RMSE validation Loss: 16.9436
Epoch 61/200, MSE training Loss: 54.4160, RMSE validation Loss: 12.0690
Epoch 71/200, MSE training Loss: 50.5677, RMSE validation Loss: 13.0393
Epoch 81/200, MSE training Loss: 50.6932, RMSE validation Loss: 12.4523
Epoch 91/200, MSE training Loss: 47.8990, RMSE validation Loss: 11.4289
Epoch 101/200, MSE training Loss: 44.5041, RMSE validation Loss: 12.9546
Epoch 111/200, MSE training Loss: 46.8019, RMSE validation Loss: 11.5754
Epoch 121/200, MSE training Loss: 45.4636, RMSE validation Loss: 11.5386
Epoch 131/200, MSE training Loss: 45.4708, RMSE validation Lo

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(range(1, 200+1), train_losses, label='Training Loss')
plt.plot(range(1, 200+1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('RNN Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Plotting training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, 200+1), train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('RNN Training Loss')
plt.legend()
plt.show()

In [ ]:
import torch
from torch.utils.data import DataLoader
import random

def predict_and_compare(model, test_loader, criterion, device):
    model.eval()  # Ensure the model is in evaluation mode
    predictions = []
    actuals = []
    total_loss = 0
    with torch.no_grad():  # No need to track gradients
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            outputs = outputs.view_as(targets)  # Ensure outputs and targets have the same shape
            loss = torch.sqrt(criterion(outputs, targets))
            total_loss += loss.item()

            predictions.extend(outputs.squeeze().tolist())
            actuals.extend(targets.tolist())

    return predictions, actuals, total_loss / len(test_loader)

def calculate_accuracy(predictions, actuals):
    accuracies = []
    for pred, actual in zip(predictions, actuals):
        if actual != 0:
            accuracy = (1 - abs(pred - actual) / abs(actual)) * 100
        else:
            accuracy = 0  # Decide how to handle actual == 0; perhaps continue or set a default accuracy value
        accuracies.append(accuracy)
    return sum(accuracies) / len(accuracies) if accuracies else 0



def print_random_predictions(predictions, actuals, num_samples=30):
    if len(predictions) < num_samples:
        num_samples = len(predictions)  # Adjust sample size if predictions are fewer than requested

    # Get random sample indices
    sample_indices = random.sample(range(len(predictions)), num_samples)

    # Print the randomly selected predictions and their actual values
    for idx in sample_indices:
        print(f"Predicted: {predictions[idx]:.4f}, Actual: {actuals[idx]:.4f}")



test_dataset = SpeedDataset('../test', sequence_length=10, overlap = 5)  
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
predictions, actuals, loss = predict_and_compare(model, test_loader, criterion, device)

# Calculate and print accuracy
accuracy = calculate_accuracy(predictions, actuals)
print(f"Average Accuracy: {accuracy:.2f}%")
print(f"Final RMSE Loss: {loss:.4f}")
print_random_predictions(predictions, actuals, 10)

In [ ]:

# Creating a plot with both predictions and actuals
plt.figure(figsize=(10, 5))

# Plotting actual speeds
plt.plot(actuals, label='Actual Speed', color='blue', marker='o', linestyle='', markersize=5)

# Plotting predicted speeds
plt.plot(predictions, label='Predicted Speed', color='red', marker='x', linestyle='', markersize=5)

plt.title('Comparison of Actual and Predicted Speeds')
plt.xlabel('Sample Index')
plt.ylabel('Speed')
plt.legend()
plt.grid(True)
plt.show()

# Scatter plot to visualize the correlation between actual and predicted speeds
plt.figure(figsize=(7, 7))
plt.scatter(actuals, predictions, color='green')
plt.title('Actual vs Predicted Speeds')
plt.xlabel('Actual Speed')
plt.ylabel('Predicted Speed')
plt.grid(True)
plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'k--')  # Diagonal line for reference
plt.show()

In [ ]:

plt.figure(figsize=(7, 7))
plt.scatter(actuals, predictions, color='green', alpha=0.5)  # Adjust transparency with alpha
plt.title('Actual vs Predicted Speeds')
plt.xlabel('Actual Speed')
plt.ylabel('Predicted Speed')
plt.grid(True)
plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'k--')  # Diagonal line for reference
plt.show()